In [2]:
import pandas as pd
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score

In [ ]:
# 가장 기본적인 뼈대 모형
def main():
    """
    메인 함수: 데이터 로드, 모델 학습 및 평가를 수행합니다.
    """
    print("스크립트 실행 시작...")

    # 1. 데이터 불러오기
    # 노트북 폴더가 아닌 src 폴더 기준이므로 '../'가 필요합니다.
    try:
        df = pd.read_csv('../data/creditcard.csv')
        print("데이터 로드 성공.")
    except FileNotFoundError:
        print("에러: '../data/creditcard.csv' 파일을 찾을 수 없습니다.")
        print("스크립트를 프로젝트의 최상위 폴더(credit-fraud-mlops)에서 실행하고 있는지 확인하세요.")
        return

    # 2. 피처(X)와 타겟(y) 분리
    X = df.drop('Class', axis=1)
    y = df['Class']
    print("피처와 타겟 분리 완료.")

    # 3. 학습용 데이터와 테스트용 데이터 분리
    # stratify=y 옵션을 사용하여 클래스 비율을 유지합니다.
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print("데이터 분리 완료.")
    print(f"학습 데이터 크기: {X_train.shape}")
    print(f"테스트 데이터 크기: {X_test.shape}")

    # 4. 모델 정의 (가장 간단한 로지스틱 회귀 모델)
    # random_state를 지정하여 실행할 때마다 동일한 결과를 얻도록 합니다.
    model = LogisticRegression(random_state=42, max_iter=1000)
    print("모델 정의 완료: LogisticRegression")

    # 5. 모델 학습
    print("모델 학습 시작...")
    model.fit(X_train, y_train)
    print("모델 학습 완료.")

    # 6. 예측 수행
    # predict_proba()를 사용하여 각 클래스에 대한 '확률'을 예측합니다.
    # AUPRC를 계산하려면 확률값이 필요합니다. [:, 1]은 사기(1)일 확률을 의미합니다.
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    print("예측 수행 완료.")

    # 7. 성능 평가 (AUPRC 계산)
    auprc = average_precision_score(y_test, y_pred_proba)
    print("\n--- 모델 평가 결과 ---")
    print(f"AUPRC (Average Precision-Recall Score): {auprc:.4f}")
    print("--------------------")

if __name__ == "__main__":
    # 이 스크립트 파일이 직접 실행될 때만 main() 함수를 호출합니다.
    main()


스크립트 실행 시작...
데이터 로드 성공.
피처와 타겟 분리 완료.
데이터 분리 완료.
학습 데이터 크기: (227845, 30)
테스트 데이터 크기: (56962, 30)
모델 정의 완료: LogisticRegression
모델 학습 시작...
모델 학습 완료.
예측 수행 완료.

--- 모델 평가 결과 ---
AUPRC (Average Precision-Recall Score): 0.7254
--------------------


c:\credit-fraud-mlops\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
# 전처리 RobustScaler, Smote 전처리 한 기본 로지스틱 
def main():
    """
    메인 함수: 데이터 로드, 전처리, 모델 학습 및 평가를 모두 수행합니다.
    """
    print("스크립트 실행 시작...")

    # 1. 데이터 불러오기
    try:
        df = pd.read_csv('../data/creditcard.csv')
        print("데이터 로드 성공.")
    except FileNotFoundError:
        print("에러: '../data/creditcard.csv' 파일을 찾을 수 없습니다.")
        return

    # --- 전처리 과정 시작 ---

    # 2. 피처 스케일링 (RobustScaler 적용)
    scaler = RobustScaler()
    df['scaled_amount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
    df['scaled_time'] = scaler.fit_transform(df['Time'].values.reshape(-1, 1))
    df.drop(['Time', 'Amount'], axis=1, inplace=True)
    print("피처 스케일링 완료.")

    # 3. 데이터 분리 (학습용/테스트용)
    X = df.drop('Class', axis=1)
    y = df['Class']
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print("데이터 분리 완료.")

    # 4. 오버샘플링 (SMOTE 적용 - 학습 데이터에만!)
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    print("오버샘플링 완료.")
    print(f"SMOTE 적용 후 학습 데이터 크기: {X_train_resampled.shape}")

    # --- 전처리 과정 종료 ---


    # 5. 모델 정의 및 학습
    model = LogisticRegression(random_state=42, max_iter=1000)
    print("모델 학습 시작...")
    # 중요: 오버샘플링된 데이터로 학습합니다!
    model.fit(X_train_resampled, y_train_resampled)
    print("모델 학습 완료.")

    # 6. 예측 및 평가
    # 중요: 평가는 원본 테스트 데이터로 수행합니다!
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auprc = average_precision_score(y_test, y_pred_proba)
    print("\n--- 모델 평가 결과 ---")
    print(f"AUPRC (Average Precision-Recall Score): {auprc:.4f}")
    print("--------------------")

if __name__ == "__main__":
    main()

스크립트 실행 시작...
데이터 로드 성공.
피처 스케일링 완료.
데이터 분리 완료.
오버샘플링 완료.
SMOTE 적용 후 학습 데이터 크기: (454902, 30)
모델 학습 시작...
모델 학습 완료.

--- 모델 평가 결과 ---
AUPRC (Average Precision-Recall Score): 0.7235
--------------------
